## Logistic Regression via PyTorch


We load a lot of libraries.  

In [68]:
import torch
print(f"""Using Torch version {torch.__version__}.  
        CUDA is {'available' if torch.cuda.is_available() else 'not available'}. 
        MPS is {'available' if torch.backends.mps.is_available() else 'not available'}""")
gpu = 'mps' if torch.backends.mps.is_available() else 'cuda'
cpu = 'cpu'
# Plotting libraries
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Label
print(f"Using bokeh version {bokeh.__version__}.")

# numpy and pandas
import numpy as np
import pandas as pd
print(f"Using pandas version {pd.__version__}.")

# tqdm makes progress bars
import tqdm
# we use train test split
from sklearn.model_selection import train_test_split

Using Torch version 2.0.0.  
        CUDA is not available. 
        MPS is available
Using bokeh version 3.1.0.
Using pandas version 2.0.0.


In [71]:
# Set up plotting to notebook; use cpu for computations
output_notebook()
device = cpu

Loading BokehJS ...

Logistic regression (for 15 features and 1 output) is a very simple neural network.

In [88]:
class LogR(torch.nn.Module):
    """A simple 15x1 logistic regression model."""
    def __init__(self):
        super().__init__()
        self.logistic=torch.nn.Linear(15,1,dtype=torch.float32)

    def forward(self,x):
        return torch.sigmoid(self.logistic(x))

We load the "food" data that we used in our earlier look at logistic regression

In [89]:
df = pd.read_csv('ifood_df.csv',delimiter=',')
features = df.columns[[0,4,5,6,7,8,9,10,11,12,13,14,24,36,37]]
df[features] = (df[features]-df[features].mean())/df[features].std()
x_train, x_test, y_train, y_test = train_test_split(df[features].values, df['Response'].values)



Create the model on the device; show the initial (randomly chosen) parameters.

In [90]:
model = LogR().to(device)
print(model)
for name,param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

LogR(
  (logistic): Linear(in_features=15, out_features=1, bias=True)
)
Layer: logistic.weight | Size: torch.Size([1, 15]) | Values : tensor([[ 0.0097, -0.1121,  0.1309,  0.0206, -0.1916,  0.0744,  0.1562, -0.2068,
         -0.1058,  0.1780, -0.2014, -0.2022, -0.0832, -0.1668,  0.1505]],
       grad_fn=<SliceBackward0>) 

Layer: logistic.bias | Size: torch.Size([1]) | Values : tensor([-0.0071], grad_fn=<SliceBackward0>) 



Set up the training data as torch tensors.  We use binary cross entropy for the loss, and stochastic gradient descent for the optimization algorithm.

In [91]:
Xt = torch.tensor(x_train,dtype=torch.float32,device=device)
Yt = torch.tensor(y_train,dtype=torch.float32,device=device)
criterion = torch.nn.functional.binary_cross_entropy
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)


In [92]:
def train(model, criterion, optimizer, Xt, Yt):
    """One step through the training loop"""
    # reset the gradient calculations
    optimizer.zero_grad()

    # forward pass
    predicted = model(Xt)
    
    # compute the loss
    loss = criterion(torch.squeeze(predicted),Yt)
    
 

    # compute the gradients by backward propogation
    loss.backward()        
        
    # adjust the weights
    optimizer.step()
    
    return loss.item()

In [93]:
def training_loop(model, data, target, learning_rate=.0001,threshold=1e-6,max_iter=100000):
    """Run the training loop and return the losses"""
    criterion = torch.nn.functional.binary_cross_entropy
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    losses = []
    prior_loss=1000000
    for i in tqdm.tqdm(range(max_iter)):
        loss = train(model,criterion,optimizer,Xt,Yt)
        losses.append(loss)
        if abs(loss-prior_loss) < threshold:
            break
        prior_loss = loss
        
    with torch.no_grad():
        prediction = model(Xt).round()
        print(f"Accuracy on training data is {((prediction - Yt.reshape(-1,1))==0).sum()/Xt.shape[0]}")
    
    return losses
    

In [94]:
def plot_loss(losses):
    """Run the model and collect the losses; return a figure"""
    
    f=figure(title=f"Loss over time",x_axis_label="Epoch",y_axis_label="Loss")
    f.line(x=list(range(len(losses))),y=losses)

    
    return f

In [96]:
model = LogR().to(device)
losses = training_loop(model, Xt, Yt)
show(plot_loss(losses))

 64%|██████████████████████████████████████████████████████████████████                                      | 63507/100000 [00:06<00:03, 9873.41it/s]


Accuracy on training data is 0.8505747318267822


Now we can check the accuracy on the test data

In [98]:
Xtst = torch.tensor(x_test,dtype=torch.float32,device=device,requires_grad=False)
Ytst = torch.tensor(y_test, dtype = torch.float32, device=device, requires_grad=False)

In [99]:
prediction = model(Xtst).round()
print(f"Accuracy is {((prediction - Ytst.reshape(-1,1))==0).sum()/Xtst.shape[0]}")
    

Accuracy is 0.8550724387168884
